In [ ]:
#!/usr/bin/env python
# coding: utf-8


In [ ]:
# # 8강 정형데이터 분류 베이스라인 모델 3 피처 중요도
# 
# - 세번째 베이스라인 모델에서 LightGBM 외에 XGBoost, CatBoost 모델링을 추가 구현해보고 각 모델의 피처 중요도를 계산하여 그려보기


In [ ]:
# ## Function 정의
# 
# - seed_everything() : seed 고정 함수
# - generate_label() : label 생성 함수
# - print_score() : 평가지표 출력 함수
# - feature_preprocessing() : 데이터 전처리 함수(label encoding, 결측치 처리)
# - feature_engineering1() : 피처 엔지니어링 함수
# - make_lgb_oof_prediction() : LightGBM cross validation out of fold train/predict 함수
# - make_xgb_oof_prediction() : XGBoost cross validation out of fold train/predict 함수
# - make_cat_oof_prediction() : CatBoost cross validation out of fold train/predict 함수
# - plot_feature_importances() : 피처 중요도 플롯 함수


In [ ]:
# ## Library Import


In [ ]:
# In[1]:


In [ ]:

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import os, sys, gc, warnings, random


In [ ]:
import datetime
import dateutil.relativedelta


In [ ]:
# Data manipulation
import pandas as pd 
import numpy as np


In [ ]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, GroupKFold
from sklearn.ensemble import RandomForestClassifier


In [ ]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool


In [ ]:
from tqdm.notebook import trange, tqdm


In [ ]:
from IPython.display import display


In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')


In [ ]:
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000


In [ ]:

# ## Fix Seed 함수 정의


In [ ]:
# In[2]:


In [ ]:

def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


In [ ]:
SEED = 42
seed_everything(SEED)


In [ ]:

# ## 레이블 생성 함수 정의
# 
# - 레이블 생성 함수는 연월을 인자로 받아서 고객 아이디별로 총 구매액이 인자로 받는 구매액 임계값을 넘는지 여부를 바이너리 레이블로 생성하는 함수


In [ ]:
# In[3]:


In [ ]:

TOTAL_THRES = 300


In [ ]:
'''
    입력인자로 받는 year_month에 대해 고객 ID별로 총 구매액이
    구매액 임계값을 넘는지 여부의 binary label을 생성하는 함수
'''
def generate_label(df, year_month, total_thres=TOTAL_THRES, print_log=False):
    df = df.copy()


In [ ]:
    # year_month에 해당하는 label 데이터 생성
    df['year_month'] = df['order_date'].dt.strftime('%Y-%m')
    df.reset_index(drop=True, inplace=True)


In [ ]:
    # year_month 이전 월의 고객 ID 추출
    cust = df[df['year_month']<year_month]['customer_id'].unique()
    # year_month에 해당하는 데이터 선택
    df = df[df['year_month']==year_month]


In [ ]:
    # label 데이터프레임 생성
    label = pd.DataFrame({'customer_id':cust})
    label['year_month'] = year_month


In [ ]:
    # year_month에 해당하는 고객 ID의 구매액의 합 계산
    grped = df.groupby(['customer_id','year_month'], as_index=False)[['total']].sum()


In [ ]:
    # label 데이터프레임과 merge하고 구매액 임계값을 넘었는지 여부로 label 생성
    label = label.merge(grped, on=['customer_id','year_month'], how='left')
    label['total'].fillna(0.0, inplace=True)
    label['label'] = (label['total'] > total_thres).astype(int)


In [ ]:
    # 고객 ID로 정렬
    label = label.sort_values('customer_id').reset_index(drop=True)
    if print_log: print(f'{year_month} - final label shape: {label.shape}')


In [ ]:
    return label


In [ ]:

# ## 평가지표 출력 함수 정의


In [ ]:
# In[4]:


In [ ]:

def print_score(label, pred, prob_thres=0.5):
    print('Precision: {:.5f}'.format(precision_score(label, pred>prob_thres)))
    print('Recall: {:.5f}'.format(recall_score(label, pred>prob_thres)))
    print('F1 Score: {:.5f}'.format(f1_score(label, pred>prob_thres)))
    print('ROC AUC Score: {:.5f}'.format(roc_auc_score(label, pred)))


In [ ]:

# ## 데이터 전처리 함수 정의
# - 범주형 변수 레이블 인코딩
# - 결측치 중위값으로 채움


In [ ]:
# In[5]:


In [ ]:

def feature_preprocessing(train, test, features, do_imputing=True):
    x_tr = train.copy()
    x_te = test.copy()


In [ ]:
    # 범주형 피처 이름을 저장할 변수
    cate_cols = []


In [ ]:
    # 레이블 인코딩
    for f in features:
        if x_tr[f].dtype.name == 'object': # 데이터 타입이 object(str)이면 레이블 인코딩
            cate_cols.append(f)
            le = LabelEncoder()
            # train + test 데이터를 합쳐서 레이블 인코딩 함수에 fit
            le.fit(list(x_tr[f].values) + list(x_te[f].values))


In [ ]:
            # train 데이터 레이블 인코딩 변환 수행
            x_tr[f] = le.transform(list(x_tr[f].values))


In [ ]:
            # test 데이터 레이블 인코딩 변환 수행
            x_te[f] = le.transform(list(x_te[f].values))


In [ ]:
    print('categorical feature:', cate_cols)


In [ ]:
    if do_imputing:
        # 중위값으로 결측치 채우기
        imputer = SimpleImputer(strategy='median')


In [ ]:
        x_tr[features] = imputer.fit_transform(x_tr[features])
        x_te[features] = imputer.transform(x_te[features])


In [ ]:
    return x_tr, x_te


In [ ]:

# ## 피처 엔지니어링 함수 정의
# - Pandas groupby aggregation 함수를 사용한 피처 엔지니어링


In [ ]:
# In[6]:


In [ ]:

def feature_engineering1(df, year_month):
    df = df.copy()


In [ ]:
    # year_month 이전 월 계산
    d = datetime.datetime.strptime(year_month, "%Y-%m")
    prev_ym = d - dateutil.relativedelta.relativedelta(months=1)
    prev_ym = prev_ym.strftime('%Y-%m')


In [ ]:
    # train, test 데이터 선택
    train = df[df['order_date'] < prev_ym]
    test = df[df['order_date'] < year_month]


In [ ]:
    # train, test 레이블 데이터 생성
    train_label = generate_label(df, prev_ym)[['customer_id','year_month','label']]
    test_label = generate_label(df, year_month)[['customer_id','year_month','label']]


In [ ]:
    # group by aggregation 함수 선언
    agg_func = ['mean','max','min','sum','count','std','skew']
    all_train_data = pd.DataFrame()


In [ ]:
    for i, tr_ym in enumerate(train_label['year_month'].unique()):
        # group by aggretation 함수로 train 데이터 피처 생성
        train_agg = train.loc[train['order_date'] < tr_ym].groupby(['customer_id']).agg(agg_func)


In [ ]:
        # 멀티 레벨 컬럼을 사용하기 쉽게 1 레벨 컬럼명으로 변경
        new_cols = []
        for col in train_agg.columns.levels[0]:
            for stat in train_agg.columns.levels[1]:
                new_cols.append(f'{col}-{stat}')


In [ ]:
        train_agg.columns = new_cols
        train_agg.reset_index(inplace = True)


In [ ]:
        train_agg['year_month'] = tr_ym


In [ ]:
        all_train_data = all_train_data.append(train_agg)


In [ ]:
    all_train_data = train_label.merge(all_train_data, on=['customer_id', 'year_month'], how='left')
    features = all_train_data.drop(columns=['customer_id', 'label', 'year_month']).columns


In [ ]:
    # group by aggretation 함수로 test 데이터 피처 생성
    test_agg = test.groupby(['customer_id']).agg(agg_func)
    test_agg.columns = new_cols


In [ ]:
    test_data = test_label.merge(test_agg, on=['customer_id'], how='left')


In [ ]:
    # train, test 데이터 전처리
    x_tr, x_te = feature_preprocessing(all_train_data, test_data, features)


In [ ]:
    print('x_tr.shape', x_tr.shape, ', x_te.shape', x_te.shape)


In [ ]:
    return x_tr, x_te, all_train_data['label'], features


In [ ]:

# ## LightGBM cross validation out of fold train/predict 함수 정의


In [ ]:
# In[7]:


In [ ]:

def make_lgb_oof_prediction(train, y, test, features, categorical_features='auto', model_params=None, folds=10):
    x_train = train[features]
    x_test = test[features]


In [ ]:
    # 테스트 데이터 예측값을 저장할 변수
    test_preds = np.zeros(x_test.shape[0])


In [ ]:
    # Out Of Fold Validation 예측 데이터를 저장할 변수
    y_oof = np.zeros(x_train.shape[0])


In [ ]:
    # 폴드별 평균 Validation 스코어를 저장할 변수
    score = 0


In [ ]:
    # 피처 중요도를 저장할 데이터 프레임 선언
    fi = pd.DataFrame()
    fi['feature'] = features


In [ ]:
    # Stratified K Fold 선언
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)


In [ ]:
    for fold, (tr_idx, val_idx) in enumerate(skf.split(x_train, y)):
        # train index, validation index로 train 데이터를 나눔
        x_tr, x_val = x_train.loc[tr_idx, features], x_train.loc[val_idx, features]
        y_tr, y_val = y[tr_idx], y[val_idx]


In [ ]:
        print(f'fold: {fold+1}, x_tr.shape: {x_tr.shape}, x_val.shape: {x_val.shape}')


In [ ]:
        # LightGBM 데이터셋 선언
        dtrain = lgb.Dataset(x_tr, label=y_tr)
        dvalid = lgb.Dataset(x_val, label=y_val)


In [ ]:
        # LightGBM 모델 훈련
        clf = lgb.train(
            model_params,
            dtrain,
            valid_sets=[dtrain, dvalid], # Validation 성능을 측정할 수 있도록 설정
            categorical_feature=categorical_features,
            verbose_eval=200
        )


In [ ]:
        # Validation 데이터 예측
        val_preds = clf.predict(x_val)


In [ ]:
        # Validation index에 예측값 저장 
        y_oof[val_idx] = val_preds


In [ ]:
        # 폴드별 Validation 스코어 출력
        print(f"Fold {fold + 1} | AUC: {roc_auc_score(y_val, val_preds)}")
        print('-'*80)


In [ ]:
        # score 변수에 폴드별 평균 Validation 스코어 저장
        score += roc_auc_score(y_val, val_preds) / folds


In [ ]:
        # 테스트 데이터 예측하고 평균해서 저장
        test_preds += clf.predict(x_test) / folds


In [ ]:
        # 폴드별 피처 중요도 저장
        fi[f'fold_{fold+1}'] = clf.feature_importance()


In [ ]:
        del x_tr, x_val, y_tr, y_val
        gc.collect()


In [ ]:
    print(f"\nMean AUC = {score}") # 폴드별 평균 Validation 스코어 출력
    print(f"OOF AUC = {roc_auc_score(y, y_oof)}") # Out Of Fold Validation 스코어 출력


In [ ]:
    # 폴드별 피처 중요도 평균값 계산해서 저장
    fi_cols = [col for col in fi.columns if 'fold_' in col]
    fi['importance'] = fi[fi_cols].mean(axis=1)


In [ ]:
    return y_oof, test_preds, fi


In [ ]:

# ## XGBoost cross validation out of fold train/predict 함수 정의


In [ ]:
# In[8]:


In [ ]:

def make_xgb_oof_prediction(train, y, test, features, model_params=None, folds=10):
    x_train = train[features]
    x_test = test[features]


In [ ]:
    # 테스트 데이터 예측값을 저장할 변수
    test_preds = np.zeros(x_test.shape[0])


In [ ]:
    # Out Of Fold Validation 예측 데이터를 저장할 변수
    y_oof = np.zeros(x_train.shape[0])


In [ ]:
    # 폴드별 평균 Validation 스코어를 저장할 변수
    score = 0


In [ ]:
    # 피처 중요도를 저장할 데이터 프레임 선언
    fi = pd.DataFrame()
    fi['feature'] = features


In [ ]:
    # Stratified K Fold 선언
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)


In [ ]:
    for fold, (tr_idx, val_idx) in enumerate(skf.split(x_train, y)):
        # train index, validation index로 train 데이터를 나눔
        x_tr, x_val = x_train.loc[tr_idx, features], x_train.loc[val_idx, features]
        y_tr, y_val = y[tr_idx], y[val_idx]


In [ ]:
        print(f'fold: {fold+1}, x_tr.shape: {x_tr.shape}, x_val.shape: {x_val.shape}')


In [ ]:
        # XGBoost 데이터셋 선언
        dtrain = xgb.DMatrix(x_tr, label=y_tr)
        dvalid = xgb.DMatrix(x_val, label=y_val)


In [ ]:
        # XGBoost 모델 훈련
        clf = xgb.train(
            model_params,
            dtrain,
            num_boost_round=10000, # 트리 개수
            evals=[(dtrain, 'train'), (dvalid, 'valid')],  # Validation 성능을 측정할 수 있도록 설정
            verbose_eval=200,
            early_stopping_rounds=100
        )


In [ ]:
        # Validation 데이터 예측
        val_preds = clf.predict(dvalid)


In [ ]:
        # Validation index에 예측값 저장 
        y_oof[val_idx] = val_preds


In [ ]:
        # 폴드별 Validation 스코어 출력
        print(f"Fold {fold + 1} | AUC: {roc_auc_score(y_val, val_preds)}")
        print('-'*80)


In [ ]:
        # score 변수에 폴드별 평균 Validation 스코어 저장
        score += roc_auc_score(y_val, val_preds) / folds


In [ ]:
        # 테스트 데이터 예측하고 평균해서 저장
        test_preds += clf.predict(xgb.DMatrix(x_test)) / folds


In [ ]:
        # 폴드별 피처 중요도 저장
        fi_tmp = pd.DataFrame.from_records([clf.get_score()]).T.reset_index()
        fi_tmp.columns = ['feature',f'fold_{fold+1}']
        fi = pd.merge(fi, fi_tmp, on='feature')


In [ ]:
        del x_tr, x_val, y_tr, y_val
        gc.collect()


In [ ]:
    print(f"\nMean AUC = {score}") # 폴드별 평균 Validation 스코어 출력
    print(f"OOF AUC = {roc_auc_score(y, y_oof)}") # Out Of Fold Validation 스코어 출력


In [ ]:
    # 폴드별 피처 중요도 평균값 계산해서 저장
    fi_cols = [col for col in fi.columns if 'fold_' in col]
    fi['importance'] = fi[fi_cols].mean(axis=1)


In [ ]:
    return y_oof, test_preds, fi


In [ ]:

# ## CatBoost cross validation out of fold train/predict 함수 정의


In [ ]:
# In[9]:


In [ ]:

def make_cat_oof_prediction(train, y, test, features, categorical_features=None, model_params=None, folds=10):
    x_train = train[features]
    x_test = test[features]


In [ ]:
    # 테스트 데이터 예측값을 저장할 변수
    test_preds = np.zeros(x_test.shape[0])


In [ ]:
    # Out Of Fold Validation 예측 데이터를 저장할 변수
    y_oof = np.zeros(x_train.shape[0])


In [ ]:
    # 폴드별 평균 Validation 스코어를 저장할 변수
    score = 0


In [ ]:
    # 피처 중요도를 저장할 데이터 프레임 선언
    fi = pd.DataFrame()
    fi['feature'] = features


In [ ]:
    # Stratified K Fold 선언
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)


In [ ]:
    for fold, (tr_idx, val_idx) in enumerate(skf.split(x_train, y)):
        # train index, validation index로 train 데이터를 나눔
        x_tr, x_val = x_train.loc[tr_idx, features], x_train.loc[val_idx, features]
        y_tr, y_val = y[tr_idx], y[val_idx]


In [ ]:
        print(f'fold: {fold+1}, x_tr.shape: {x_tr.shape}, x_val.shape: {x_val.shape}')


In [ ]:
        # CatBoost 모델 훈련
        clf = CatBoostClassifier(**model_params)
        clf.fit(x_tr, y_tr,
                eval_set=(x_val, y_val), # Validation 성능을 측정할 수 있도록 설정
                cat_features=categorical_features,
                use_best_model=True,
                verbose=True)


In [ ]:
        # Validation 데이터 예측
        val_preds = clf.predict_proba(x_val)[:,1]


In [ ]:
        # Validation index에 예측값 저장 
        y_oof[val_idx] = val_preds


In [ ]:
        # 폴드별 Validation 스코어 출력
        print(f"Fold {fold + 1} | AUC: {roc_auc_score(y_val, val_preds)}")
        print('-'*80)


In [ ]:
        # score 변수에 폴드별 평균 Validation 스코어 저장
        score += roc_auc_score(y_val, val_preds) / folds


In [ ]:
        # 테스트 데이터 예측하고 평균해서 저장
        test_preds += clf.predict_proba(x_test)[:,1] / folds


In [ ]:
        # 폴드별 피처 중요도 저장
        fi[f'fold_{fold+1}'] = clf.feature_importances_


In [ ]:
        del x_tr, x_val, y_tr, y_val
        gc.collect()


In [ ]:
    print(f"\nMean AUC = {score}") # 폴드별 평균 Validation 스코어 출력
    print(f"OOF AUC = {roc_auc_score(y, y_oof)}") # Out Of Fold Validation 스코어 출력


In [ ]:
    # 폴드별 피처 중요도 평균값 계산해서 저장
    fi_cols = [col for col in fi.columns if 'fold_' in col]
    fi['importance'] = fi[fi_cols].mean(axis=1)


In [ ]:
    return y_oof, test_preds, fi


In [ ]:

# ## 피처 중요도 플롯 함수 정의


In [ ]:
# In[10]:


In [ ]:

def plot_feature_importances(df, n=20, color='blue', figsize=(12,8)):
    # 피처 중요도 순으로 내림차순 정렬
    df = df.sort_values('importance', ascending = False).reset_index(drop = True)


In [ ]:
    # 피처 중요도 정규화 및 누적 중요도 계산
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])


In [ ]:
    plt.rcParams['font.size'] = 12
    plt.style.use('fivethirtyeight')
    # 피처 중요도 순으로 n개까지 바플롯으로 그리기
    df.loc[:n, :].plot.barh(y='importance_normalized', 
                            x='feature', color=color, 
                            edgecolor='k', figsize=figsize,
                            legend=False)


In [ ]:
    plt.xlabel('Normalized Importance', size=18); plt.ylabel(''); 
    plt.title(f'Top {n} Most Important Features', size=18)
    plt.gca().invert_yaxis()


In [ ]:
    return df


In [ ]:

# ## Train 데이터 읽기
# - 9개의 feature를 가진 780502개의 train data


In [ ]:
# In[11]:


In [ ]:

data = pd.read_csv("../input/train.csv", parse_dates=["order_date"])
print(data.shape) # data shape 확인
data.head()


In [ ]:

# ## Pandas info() 함수로 데이터 타입 및 null 체크


In [ ]:
# In[12]:


In [ ]:

data.info()


In [ ]:

# ## Pandas isna(), sum() 함수로 null 데이터 개수 체크


In [ ]:
# In[13]:


In [ ]:

data.isna().sum()


In [ ]:

# ## Pandas describe() 함수에 include='all' 인자 설정으로 수치형, 범주형 데이터 기본 통계량 확인


In [ ]:
# In[14]:


In [ ]:

data.describe(include='all')


In [ ]:

# ## 2011년 11월 Label 데이터 생성


In [ ]:
# In[15]:


In [ ]:

label_2011_11 = generate_label(data, '2011-11')['label']


In [ ]:

# ## Label 데이터 분포 플롯


In [ ]:
# In[16]:


In [ ]:

sns.countplot(label_2011_11);
label_2011_11.value_counts()


In [ ]:

# ## LightGBM Hyperparameter 선언
# - early stopping 기능을 사용하기 위해 n_estimators, early_stopping_rounds 설정


In [ ]:
# In[17]:


In [ ]:

model_params = {
    'objective': 'binary', # 이진 분류
    'boosting_type': 'gbdt',
    'metric': 'auc', # 평가 지표 설정
    'feature_fraction': 0.8, # 피처 샘플링 비율
    'bagging_fraction': 0.8, # 데이터 샘플링 비율
    'bagging_freq': 1,
    'n_estimators': 10000, # 트리 개수
    'early_stopping_rounds': 100,
    'seed': SEED,
    'verbose': -1,
    'n_jobs': -1,    
}


In [ ]:

# ## 2011년 12월 LightGBM 베이스라인 모델 예측
# - Validation 성능 측정


In [ ]:
# In[18]:


In [ ]:

train, test, y, features = feature_engineering1(data, '2011-12')
y_oof, test_preds, fi = make_lgb_oof_prediction(train, y, test, features, model_params=lgb_params)


In [ ]:

# ## 2011년 12월 LightGBM 베이스라인 모델 예측 데이터 분포


In [ ]:
# In[19]:


In [ ]:

sns.distplot(test_preds)
plt.show()


In [ ]:

# ## LightGBM 피처 중요도 플롯


In [ ]:
# In[20]:


In [ ]:

fi = plot_feature_importances(fi)


In [ ]:

# ## XGBoost Hyperparameter 선언


In [ ]:
# In[21]:


In [ ]:

xgb_params = {
    'objective': 'binary:logistic', # 이진 분류
    'learning_rate': 0.1, # 학습률
    'max_depth': 6, # 트리 최고 깊이
    'colsample_bytree': 0.8, # 피처 샘플링 비율
    'subsample': 0.8, # 데이터 샘플링 비율
    'eval_metric': 'auc', # 평가 지표 설정
    'seed': SEED,
} 


In [ ]:

# ## 2011년 12월 XGBoost 베이스라인 모델 예측
# - Validation 성능 측정


In [ ]:
# In[22]:


In [ ]:

y_oof, test_preds, fi_xgb = make_xgb_oof_prediction(train, y, test, features, model_params=xgb_params)


In [ ]:

# ## 2011년 12월 XGBoost 베이스라인 모델 예측 데이터 분포


In [ ]:
# In[23]:


In [ ]:

sns.distplot(test_preds)
plt.show()


In [ ]:

# ## XGBoost 피처 중요도 플롯


In [ ]:
# In[24]:


In [ ]:

fi_xgb = plot_feature_importances(fi_xgb)


In [ ]:

# ## CatBoost Hyperparameter 선언


In [ ]:
# In[25]:


In [ ]:

cat_params = {
    'n_estimators': 10000, # 트리 개수
    'learning_rate': 0.07, # 학습률
    'eval_metric': 'AUC', # 평가 지표 설정
    'loss_function': 'Logloss', # 손실 함수 설정
    'random_seed': SEED,
    'metric_period': 100,
    'od_wait': 100, # early stopping round
    'depth': 6, # 트리 최고 깊이
    'rsm': 0.8, # 피처 샘플링 비율
}


In [ ]:

# ## 2011년 12월 CatBoost 베이스라인 모델 예측
# - Validation 성능 측정


In [ ]:
# In[26]:


In [ ]:

y_oof, test_preds, fi_cat = make_cat_oof_prediction(train, y, test, features, model_params=cat_params)


In [ ]:

# ## 2011년 12월 CatBoost 베이스라인 모델 예측 데이터 분포


In [ ]:
# In[27]:


In [ ]:

sns.distplot(test_preds)
plt.show()


In [ ]:

# ## CatBoost 피처 중요도 플롯


In [ ]:
# In[28]:


In [ ]:

fi_cat = plot_feature_importances(fi_cat)
